In [3]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize 

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### Loading Data 

Import json files and convert to dataframe

** better to use json.load or pandas.read_json?

In [ ]:
# input_file = open('/opendota api constants/heroes.json')
# hero_id = json.load(input_file)
# input_file.close()
# print('hero_id')

In [26]:
### cleaning with starladder data 

input_file = open('matches/league_id_10979_starladder_s2_matchdata.json') 
data = json.load(input_file)
input_file.close()

In [5]:
#data = pd.read_json('matches/league_id_10979_starladder_s2_matchdata.json')

In [27]:
big_df = json_normalize(data)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



### Create Dataframe for match results

This will cover general id columns and basic match results such as match id, league id, dire/radiant teams, and if radiant won

Method 1: since data is saved as json, use pd.read_json

In [71]:
#data2 = pd.read_json('matches/league_id_10749_ti9_matchdata.json')
#data2.head(1)

,match_id,barracks_status_dire,barracks_status_radiant,chat,cluster,cosmetics,dire_score,dire_team_id,draft_timings,duration,engine,first_blood_time,game_mode,human_players,leagueid,lobby_type,match_seq_num,negative_votes,objectives,picks_bans,positive_votes,radiant_gold_adv,radiant_score,radiant_team_id,radiant_win,radiant_xp_adv,skill,start_time,teamfights,tower_status_dire,tower_status_radiant,version,replay_salt,series_id,series_type,league,radiant_team,dire_team,players,patch,all_word_counts,my_word_counts,throw,loss,replay_url,region,comeback,stomp
0,4968150000,0,63,"[{'time': -960, 'type': 'chatwheel', 'key': '1...",308,"{'647': 130, '4670': 130, '4794': 4, '5181': 1...",42,36.0,"[{'order': 1, 'pick': False, 'active_team': 2,...",4934,1,29,2,10,10749,1,4167421134,369,"[{'time': 29, 'type': 'CHAT_MESSAGE_FIRSTBLOOD...","[{'is_pick': False, 'hero_id': 106, 'team': 1,...",4918,"[0, 495, 762, 808, 878, 1145, 2106, 2059, 2167...",54,39.0,True,"[0, 394, 247, 561, 224, 839, 1582, 1728, 1771,...",NaN,2019-08-15 11:20:48,"[{'start': 915, 'end': 959, 'last_death': 944,...",0,1796,21,37509985,358264,1,"{'leagueid': 10749, 'ticket': None, 'banner': ...","{'team_id': 39, 'name': 'Evil Geniuses', 'tag'...","{'team_id': 36, 'name': 'Natus Vincere', 'tag'...","[{'match_id': 4968150000, 'player_slot': 0, 'a...",41,"{'gl': 1, 'hf': 1, 'gg': 5, 'wp': 1}",{},9155.0,18713.0,http://replay308.valve.net/570/4968150000_3750...,NaN,NaN,NaN


In [28]:
match_df = big_df[['match_id','duration', 'leagueid','match_seq_num','series_id', 'series_type', 'patch', 'dire_team_id', 'radiant_team_id', 'radiant_win' ]]
match_df.info()
#match_df.isnull

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   match_id         153 non-null    int64
 1   duration         153 non-null    int64
 2   leagueid         153 non-null    int64
 3   match_seq_num    153 non-null    int64
 4   series_id        153 non-null    int64
 5   series_type      153 non-null    int64
 6   patch            153 non-null    int64
 7   dire_team_id     153 non-null    int64
 8   radiant_team_id  153 non-null    int64
 9   radiant_win      153 non-null    bool 
dtypes: bool(1), int64(9)
memory usage: 11.0 KB


In [16]:
groupedDF = match_df.filter(items=['match_id','radiant_win',
                                          'patch'])  # Remove columns
groupedDF = groupedDF.groupby(['patch']).count()  # Perform Aggregate
print(groupedDF)

       match_id  radiant_win
patch                       
40          108          108
41           45           45


In [29]:
matches = match_df[match_df.patch == 41]

In [104]:
#match_df.sort_values(by='match_id', ascending=True)

## Method 2: load json file and normalize using pandas.json_normalize.

** Dunno if there is any significant advantage to doing it one way or the other. 

In [12]:
#json_struct = json.loads(data.to_json(orient="records"))    
#df_flat = pd.io.json.json_normalize(json_struct) #use pd.io.json

In [31]:
picks_df = json_normalize(data, 'picks_bans', max_level=0)
#picks_df.head(100)
#picks_df.shape

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



In [32]:
df = picks_df[['match_id', 'order', 'is_pick', 'team', 'hero_id']]

In [33]:
df = df.astype(str)
df = df.groupby('match_id')['is_pick','team','hero_id'].agg(' '.join)
#df.head(100)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [34]:
#new = df['order'].str.split(' ', expand=True)
new1 = df['is_pick'].str.split(' ', expand=True)
new2 = df['team'].str.split(' ', expand=True)
new3 = df['hero_id'].str.split(' ', expand=True)
#new1, new2, new3

In [35]:
# Get column labels for the columns 
total_picks = 21 # recall range is 0-21 = 22 picks
new1.columns = ['1is_pick', '2is_pick', '3is_pick', '4is_pick', '5is_pick', '6is_pick', '7is_pick','8is_pick','9is_pick','10is_pick','11is_pick','12is_pick','13is_pick','14is_pick','15is_pick','16is_pick','17is_pick','18is_pick','19is_pick','20is_pick','21is_pick', '22is_pick']
new2.columns = ['1team', '2team', '3team', '4team', '5team', '6team', '7team', '8team', '9team', '10team', '11team', '12team', '13team', '14team', '15team', '16team', '17team', '18team', '19team', '20team', '21team', '22team']
new3.columns = ['1hero_id', '2hero_id', '3hero_id', '4hero_id', '5hero_id', '6hero_id', '7hero_id', '8hero_id', '9hero_id', '10hero_id', '11hero_id', '12hero_id', '13hero_id', '14hero_id', '15hero_id', '16hero_id', '17hero_id', '18hero_id', '19hero_id', '20hero_id', '21hero_id',
               '22hero_id']

In [36]:
data = new1.join(new2).join(new3)
#picks_df = new1.join(new2).join(new3)
#picks_df.dtypes
#picks_df.head(100)
#data.shape

Quick eyeball inspection to check if the draft information is preserved:


Comparing match_id = 4888646940, we can see from above that the first pick (order=0) information was: 
- first pick (1ban): '4888646940	0	False	1	66'; w/c lines up with 1is_pick, 1hero_id, etc...
- last pick: 9106	4888646940	21	True	0	67; w/c lines up with 22is_pick, 22hero_id, etc...

So, at least by a quick visual inspection, seems like the drafting data has been successfully cleaned. 

### Replacing T/F values

Convert T/F to boolean 

In [41]:
# source: https://github.com/codexponent/dota2-draft-analysis/blob/master/draft-ml.ipynb
# # # Converting boolean value into number
boolean_values = 22
for i in range(boolean_values):
    pick_id = '{}is_pick'.format(i+1)
    data[pick_id] = data[pick_id].apply(lambda x: 1 if x else 0)

Re-organize columns according to prefix order:
- pretty sure there's a better way to do this 

In [37]:
picks_df = data[['1is_pick', '1team', '1hero_id',
               '2is_pick', '2team', '2hero_id', 
               '3is_pick', '3team', '3hero_id', 
               '4is_pick', '4team', '4hero_id', 
               '5is_pick', '5team', '5hero_id', 
               '6is_pick', '6team', '6hero_id', 
               '7is_pick', '7team', '7hero_id', 
               '8is_pick', '8team', '8hero_id', 
               '9is_pick', '9team', '9hero_id', 
               '10is_pick', '10team', '10hero_id', 
               '11is_pick', '11team', '11hero_id', 
               '12is_pick', '12team', '12hero_id',
               '13is_pick', '13team', '13hero_id', 
               '14is_pick', '14team', '14hero_id', 
               '15is_pick', '15team', '15hero_id', 
               '16is_pick', '16team', '16hero_id', 
               '17is_pick', '17team', '17hero_id',
               '18is_pick', '18team', '18hero_id', 
               '19is_pick', '19team', '19hero_id',
               '20is_pick', '20team', '20hero_id',
               '21is_pick', '21team', '21hero_id', 
                '22is_pick', '22team', '22hero_id']]

In [39]:
picks = picks_df
#matchhistory = match_df

In [44]:
matches2 = matches[['match_id', 'radiant_win']]

In [45]:
picks_clean = picks.copy()

In [53]:
cpicks = picks_clean.reset_index()

In [50]:
#p = picks_df[['match_id']] 
#m = matches_df.copy()



ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [54]:
# picks dictionary
picks_dict = {'match_id': int, '1is_pick': bool, '1team': bool, '1hero_id': int,
               '2is_pick': bool, '2team': bool, '2hero_id': int, 
               '3is_pick': bool, '3team': bool, '3hero_id': int, 
               '4is_pick': bool, '4team': bool, '4hero_id': int, 
               '5is_pick': bool, '5team': bool, '5hero_id': int, 
               '6is_pick': bool, '6team': bool, '6hero_id': int, 
               '7is_pick': bool, '7team': bool, '7hero_id': int, 
               '8is_pick': bool, '8team': bool, '8hero_id': int, 
               '9is_pick': bool, '9team': bool, '9hero_id': int, 
               '10is_pick': bool, '10team':bool, '10hero_id': int, 
               '11is_pick': bool, '11team':bool, '11hero_id': int, 
               '12is_pick': bool, '12team':bool, '12hero_id': int,
               '13is_pick': bool, '13team':bool, '13hero_id': int, 
               '14is_pick': bool, '14team':bool, '14hero_id': int, 
               '15is_pick': bool, '15team':bool, '15hero_id': int, 
               '16is_pick': bool, '16team':bool, '16hero_id': int, 
               '17is_pick': bool, '17team':bool, '17hero_id': int,
               '18is_pick': bool, '18team':bool, '18hero_id': int, 
               '19is_pick': bool, '19team':bool, '19hero_id': int,
               '20is_pick': bool, '20team':bool, '20hero_id': int,
               '21is_pick': bool, '21team':bool, '21hero_id': int, 
                '22is_pick': bool, '22team':bool, '22hero_id': int}

cpicks = cpicks.astype(picks_dict)

In [61]:
## picks_cleaned (44,68), and correct data types
picks_cleaned = matches2.merge(cpicks, on=['match_id'])
picks_cleaned.to_csv('starladder_picks_cleaned_merged.csv')
matches.to_csv('starladder_matches_cleaned_merged.csv')

In [44]:
picks.to_csv('data/starladder_picks.csv')
matchhistory.to_csv('data/starladder_matchhistory.csv') 

In [1]:
picks.info()

NameError: name 'picks' is not defined

In [2]:
picks2 = pd.read_csv('GET-EM/EDA/opendota_matches/data/ti9_picks_cleaned.csv')

NameError: name 'pd' is not defined